<a href="https://colab.research.google.com/github/laloflogar/RB2026/blob/main/Gr%C3%A1ficas_Internacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gráfica 1

Descargar los datos de la FRED.

In [23]:
import requests
import pandas as pd
import numpy as np

def descarga_fred_series(series_dict, fechainicio, fechafin):
    # Token de acceso a la API de FRED
    api_key = 'b24da1f1e67d32a43455a8d8ba7d4d53'
    base_url = 'https://api.stlouisfed.org/fred/series/observations'

    all_data = []

    for serie, nombre in series_dict.items():
        # Parámetros de la solicitud
        params = {
            'series_id': serie,
            'api_key': api_key,
            'file_type': 'json',
            'observation_start': fechainicio,
            'observation_end': fechafin
        }

        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            print(f'Error en la consulta para la serie {nombre}, código {response.status_code}')
            continue

        raw_data = response.json()

        # FRED devuelve los datos en la llave 'observations'
        if 'observations' in raw_data and len(raw_data['observations']) > 0:
            data = raw_data['observations']
            df = pd.DataFrame(data)

            # Limpieza de datos: FRED usa "." para valores faltantes
            df['value'] = df['value'].replace('.', np.nan)
            df['value'] = df['value'].astype(float)

            # Formateo de fechas
            df['date'] = pd.to_datetime(df['date'])
            df.set_index('date', inplace=True)

            # Seleccionar solo la columna de valor y renombrarla
            df = df[['value']].rename(columns={'value': nombre})
            all_data.append(df)
        else:
            print(f"No se encontraron datos para la serie {nombre}")

    if all_data:
        # Unimos todas las series por su fecha (índice)
        df_final = pd.concat(all_data, axis=1)
        return df_final
    else:
        print("No se descargaron datos.")
        return None

# --- Nómina no agrícola ---

series_fred = {
    'PAYEMS': 'Nómina no agrícola'
}

fechainicio = '2019-01-01'
fechafin = pd.Timestamp.today().strftime('%Y-%m-%d')

datos = descarga_fred_series(series_fred, fechainicio, fechafin)

if datos is not None:
    print("Datos de FRED descargados exitosamente ✅ :")
    print(datos.tail())

Datos de FRED descargados exitosamente ✅ :
            Nómina no agrícola
date                          
2025-08-01            159485.0
2025-09-01            159593.0
2025-10-01            159420.0
2025-11-01            159476.0
2025-12-01            159526.0


Transformar el Dataframe

In [25]:
var_mensual = datos['Nómina no agrícola'].diff().to_frame(name='Variación mensual (Nómina no agrícola)')
print(var_mensual.tail())

            Variación mensual (Nómina no agrícola)
date                                              
2025-08-01                                   -26.0
2025-09-01                                   108.0
2025-10-01                                  -173.0
2025-11-01                                    56.0
2025-12-01                                    50.0


Gráfica ##

In [90]:
import plotly.express as px

# Crear la figura
fig = px.bar(
    var_mensual,
    x=var_mensual.index,
    y='Variación mensual (Nómina no agrícola)',
    title='<b>Gráfica 00</b><br><b>EUA: nómina no agrícola</b><br><sup>Variación mensual en miles de pesos, a.e.</sup>',
    width=850,  # Ancho en píxeles
    height=500
)


#Dar estilo a la figura
fig.update_layout(
    plot_bgcolor='white', # Fondo blanco para el área del gráfico
    paper_bgcolor='white', # Fondo blanco para el área del papel
    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=False,
        title_text='',
        tickfont=dict(color="black")
    ),
    yaxis=dict(
        title_text=' ',
        range=[-500, 1000],
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=False,
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=1,
        title_font=dict(color="black"),
        tickfont=dict(color="black")
    ),
    title_x=0.5, #Título centrado

# Notas al pie
annotations=[
        dict(
            xref="paper", yref="paper",
            x=0, y=-0.1,
            xanchor="left", yanchor="top",
            text="a.e. / cifras con ajuste estacional<br>Nota: se acotó el rango del eje para facilitar su lectura<br>Fuente: Elaboración propia con datos de FRED.",
            align="left",
            font=dict(size=12),
            showarrow=False
        ),
#Acotaciones inferiores
        dict(
            xref="paper", yref="paper",
            x=0.16, y=0,
            text="",
            showarrow=True,
            arrowhead=0,
            arrowsize=1,
            arrowwidth=3,
            arrowcolor="red",
            ax=25, ay=-5,
            xanchor="left", yanchor="bottom"
        ),
        dict(
            xref="paper", yref="paper",
            x=0.16, y=0.01,
            text="",
            showarrow=True,
            arrowhead=0,
            arrowsize=1,
            arrowwidth=5,
            arrowcolor="white",
            ax=25, ay=-5,
            xanchor="left", yanchor="bottom"
        ),
      dict(
            xref="paper", yref="paper",
            x=0.16, y=0.02,
            text="",
            showarrow=True,
            arrowhead=0,
            arrowsize=1,
            arrowwidth=2.5,
            arrowcolor="red",
            ax=25, ay=-5,
            xanchor="left", yanchor="bottom"
        ),
#Acotaciones superiores
        dict(
            xref="paper", yref="paper",
            x=0.18, y=0.98,
            text="",
            showarrow=True,
            arrowhead=0,
            arrowsize=1,
            arrowwidth=3,
            arrowcolor="red",
            ax=50, ay=-5,
            xanchor="left", yanchor="bottom"
        ),
        dict(
            xref="paper", yref="paper",
            x=0.18, y=0.99,
            text="",
            showarrow=True,
            arrowhead=0,
            arrowsize=1,
            arrowwidth=5,
            arrowcolor="white",
            ax=50, ay=-5,
            xanchor="left", yanchor="bottom"
        ),
      dict(
            xref="paper", yref="paper",
            x=0.18, y=1,
            text="",
            showarrow=True,
            arrowhead=0,
            arrowsize=1,
            arrowwidth=2.5,
            arrowcolor="red",
            ax=50, ay=-5,
            xanchor="left", yanchor="bottom"
        )
    ]

)

# Establecer el color de las barras a azul
fig.update_traces(marker_color='blue')

#Mostrar la figura
fig.show()

# Gráfica 2

In [119]:
import requests
import pandas as pd
import numpy as np

def descarga_fred_series(series_dict, fechainicio, fechafin):
    # Token de acceso a la API de FRED
    api_key = 'b24da1f1e67d32a43455a8d8ba7d4d53'
    base_url = 'https://api.stlouisfed.org/fred/series/observations'

    all_data = []

    for serie, nombre in series_dict.items():
        # Parámetros de la solicitud
        params = {
            'series_id': serie,
            'api_key': api_key,
            'file_type': 'json',
            'observation_start': fechainicio,
            'observation_end': fechafin
        }

        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            print(f'Error en la consulta para la serie {nombre}, código {response.status_code}')
            continue

        raw_data = response.json()

        # FRED devuelve los datos en la llave 'observations'
        if 'observations' in raw_data and len(raw_data['observations']) > 0:
            data = raw_data['observations']
            df = pd.DataFrame(data)

            # Limpieza de datos: FRED usa "." para valores faltantes
            df['value'] = df['value'].replace('.', np.nan)
            df['value'] = df['value'].astype(float)

            # Formateo de fechas
            df['date'] = pd.to_datetime(df['date'])
            df.set_index('date', inplace=True)

            # Seleccionar solo la columna de valor y renombrarla
            df = df[['value']].rename(columns={'value': nombre})
            all_data.append(df)
        else:
            print(f"No se encontraron datos para la serie {nombre}")

    if all_data:
        # Unimos todas las series por su fecha (índice)
        df_final = pd.concat(all_data, axis=1)
        return df_final
    else:
        print("No se descargaron datos.")
        return None

# --- EUA: PIB real y sus componentes ---

series_fred = {
    'GDPC1': 'Producto Interno Bruto',
    'PCECC96': 'Consumo privado',
    'PRFIC1': 'Inversión residencial',
    'PNFIC1': 'Inversión no residencial',
    'CBIC1': 'Inventarios',
    'NETEXC': 'Exportaciones netas',
    'GCEC1': 'Gasto público'
}

fechainicio = '2020-12-01'
fechafin = pd.Timestamp.today().strftime('%Y-%m-%d')

datos = descarga_fred_series(series_fred, fechainicio, fechafin)

if datos is not None:
    print("Datos de FRED descargados exitosamente ✅ :")
    print(datos.tail())

Datos de FRED descargados exitosamente ✅ :
            Producto Interno Bruto  Consumo privado  Inversión residencial  \
date                                                                         
2024-07-01               23478.570        16165.768                782.472   
2024-10-01               23586.542        16320.890                790.685   
2025-01-01               23548.210        16345.793                788.779   
2025-04-01               23770.976        16445.685                778.530   
2025-07-01               24024.957        16589.126                768.301   

            Inversión no residencial  Inventarios  Exportaciones netas  \
date                                                                     
2024-07-01                  3547.627       69.426            -1064.895   
2024-10-01                  3514.424       17.053            -1068.994   
2025-01-01                  3595.389      171.969            -1380.718   
2025-04-01                  3659.320    

Transformar el Dataframe

In [126]:
datos['PIB(%)'] = datos['Producto Interno Bruto'].pct_change() * 100
datos['CP(%)'] = (datos['Consumo privado'].diff()/datos['Producto Interno Bruto'].diff())*datos['PIB(%)']
datos['IR(%)'] = (datos['Inversión residencial'].diff()/datos['Producto Interno Bruto'].diff())*datos['PIB(%)']
datos['INR(%)'] = (datos['Inversión no residencial'].diff()/datos['Producto Interno Bruto'].diff())*datos['PIB(%)']
datos['I(%)'] = (datos['Inventarios'].diff()/datos['Producto Interno Bruto'].diff())*datos['PIB(%)']
datos['NX(%)'] = (datos['Exportaciones netas'].diff()/datos['Producto Interno Bruto'].diff())*datos['PIB(%)']
datos['GP(%)'] = (datos['Gasto público'].diff()/datos['Producto Interno Bruto'].diff())*datos['PIB(%)']

# Redondeo
percentage_columns = ['PIB(%)', 'CP(%)', 'IR(%)', 'INR(%)', 'I(%)', 'NX(%)', 'GP(%)']
datos[percentage_columns] = datos[percentage_columns].round(1)

print('Datos con tasa de crecimiento del PIB:')
print(datos.tail())

Datos con tasa de crecimiento del PIB:
            Producto Interno Bruto  Consumo privado  Inversión residencial  \
date                                                                         
2024-07-01               23478.570        16165.768                782.472   
2024-10-01               23586.542        16320.890                790.685   
2025-01-01               23548.210        16345.793                788.779   
2025-04-01               23770.976        16445.685                778.530   
2025-07-01               24024.957        16589.126                768.301   

            Inversión no residencial  Inventarios  Exportaciones netas  \
date                                                                     
2024-07-01                  3547.627       69.426            -1064.895   
2024-10-01                  3514.424       17.053            -1068.994   
2025-01-01                  3595.389      171.969            -1380.718   
2025-04-01                  3659.320      -1

Graficar

In [167]:
import plotly.graph_objects as go

# Filtrar datos a partir del primer trimestre de 2021
datos_filtrados = datos[datos.index >= '2021-01-01']

#Eje x multinivel
mapa_romanos = {1: "I", 2: "II", 3: "III", 4: "IV"}
años = datos_filtrados.index.year
trimestres = [mapa_romanos[q] for q in datos_filtrados.index.quarter]
eje_x_multinivel = [años, trimestres]

#Crear figura
fig = go.Figure()

fig.add_trace(go.Scatter(x=eje_x_multinivel, y=datos_filtrados["PIB(%)"], name="PIB"))
fig.add_trace(go.Bar(x=eje_x_multinivel, y=datos_filtrados["CP(%)"], name="Consumo privado"))
fig.add_trace(go.Bar(x=eje_x_multinivel, y=datos_filtrados["IR(%)"], name="Inversión residencial"))
fig.add_trace(go.Bar(x=eje_x_multinivel, y=datos_filtrados["INR(%)"], name="Inversión no residencial"))
fig.add_trace(go.Bar(x=eje_x_multinivel, y=datos_filtrados["I(%)"], name="Inventarios"))
fig.add_trace(go.Bar(x=eje_x_multinivel, y=datos_filtrados["NX(%)"], name="Exportaciones netas"))
fig.add_trace(go.Bar(x=eje_x_multinivel, y=datos_filtrados["GP(%)"], name="Gasto público"))

fig.update_layout(barmode='relative',
 title_text='<b>Gráfica 00</b><br><b>EUA: PIB real y componentes</b><br><sup>Variación trimestral en por ciento y contribuciones en puntos porcentuales, a.e.</sup>',
 width=850,
 height=500
)


#Dar estilo a la figura
fig.update_layout(
    plot_bgcolor='white', # Fondo blanco para el área del gráfico
    paper_bgcolor='white', # Fondo blanco para el área del papel
    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=False,
        title_text='',
        tickfont=dict(color="black")
    ),
    yaxis=dict(
        title_text=' ',
        range=[-2, 3],
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=False,
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=1,
        title_font=dict(color="black"),
        tickfont=dict(color="black")
    ),
    title_x=0.5, #Título centrado

# Notas al pie
annotations=[
        dict(
            xref="paper", yref="paper",
            x=0, y=-0.12,
            xanchor="left", yanchor="top",
            text="a.e. / cifras con ajuste estacional<br>Fuente: Elaboración propia con datos de FRED.",
            align="left",
            font=dict(size=12),
            showarrow=False
        )
      ]

)

# Define color mapping
color_map = {
    "PIB": "black",
    "Consumo privado": "red",
    "Inversión residencial": "blue",
    "Inversión no residencial": "green",
    "Inventarios": "yellow",
    "Exportaciones netas": "aquamarine",
    "Gasto público": "purple"
}

# Apply colors to traces
for trace in fig.data:
    if trace.name in color_map:
        if trace.type == 'bar':
            trace.marker.color = color_map[trace.name]
        elif trace.type == 'scatter':
            trace.line.color = color_map[trace.name]

fig.show()

# Gráfica 3